In [1]:
% run 1-datasource.ipynb

In [2]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import scipy.stats as sc
import torch.nn.functional as F
from skorch.net import NeuralNet,train_loss_score,valid_loss_score
from skorch.callbacks import EpochTimer,EpochScoring,PrintLog
from sklearn.preprocessing import StandardScaler

In [3]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [4]:
for _, i in icebergs.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [5]:
y = icebergs.is_iceberg.values.astype(np.float32)
x = (
    np.stack(
        [
            scaler_1.transform(np.stack(icebergs.band_1)),
            scaler_2.transform(np.stack(icebergs.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).astype(np.float32)

In [6]:
x.dtype, y.dtype

(dtype('float32'), dtype('float32'))

In [7]:
type(x), type(y)

(numpy.ndarray, numpy.ndarray)

In [8]:
y.shape, x.shape

((1604,), (1604, 2, 75, 75))

In [32]:
insize1 = 2 
#outsize1 = 32 
#outsize2 = 32

class Net(nn.Module):
    def __init__(self,outsize1=16,outsize2=32):
        super().__init__()
        
        #self.padding = padding
        #self.kernel1 = kernel1
        #self.kernel2 = kernel2 
        self.outsize1= outsize1
        self.outsize2 = outsize2
        #self.stride = stride
        #self.group? 
        #insize outsize ?
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(insize1, outsize1, kernel_size=7, stride=1, padding=2, groups=2),
            nn.BatchNorm2d(outsize1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(outsize1, outsize2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(outsize2*81, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [33]:
net = NeuralNet(
    Net,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, #regularization
    max_epochs=150,
    batch_size=25,
    lr=0.000001,
    callbacks= [  
            ('valid_log_loss', EpochScoring(
                'neg_log_loss',
                name='valid_log_loss',
                lower_is_better=False,
            ))
            ],
    use_cuda = True
)

In [34]:
import scipy.stats as sc
param_dis = {
    'batch_size': [32,64],
    'module__outsize1':[16,32,64],
    'module__outsize2':[16,32,64],
}

In [35]:
from sklearn.model_selection import RandomizedSearchCV
gsR = RandomizedSearchCV(net, param_dis, refit=False,scoring='f1',n_jobs=1,n_iter=5)

In [ ]:
gsR.fit(x,y)

  epoch    train_loss    valid_log_loss    valid_loss     dur
-------  ------------  ----------------  ------------  ------
      1        0.6866           -9.6534        0.6876  1.7003
      2        0.6811          -16.4624        0.6809  1.3273
      3        0.6772          -16.4624        0.6780  1.3111
      4        0.6741          -16.4624        0.6757  1.3312
      5        0.6715          -16.4624        0.6737  1.3133
      6        0.6692          -16.4624        0.6717  1.3061
      7        0.6671          -16.4624        0.6698  1.3050
      8        0.6652          -16.4624        0.6679  1.3086
      9        0.6634          -16.4624        0.6659  1.3065
     10        0.6616          -16.4624        0.6640  1.3016
     11        0.6600          -16.4624        0.6621  1.3062
     12        0.6584          -16.4624        0.6602  1.3250
     13        0.6568          -16.4624        0.6583  1.3066
     14        0.6553          -16.4624        0.6565  1.3057
     15 

In [14]:
gsR.best_params_,gsR.best_score_

({'batch_size': 32, 'module__outsize1': 16, 'module__outsize2': 16}, 0.0)